Open the list of validation glaciers that we have, and then order every single Landsat 5,7,8,9 and Sentinel-2 image of it from GEE

In [7]:
import os
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import ee

# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")

# open list of validation images
all_validation_df = pd.read_csv(os.path.join(folder_AGVA, 'Validation', 'Validation Glaciers.csv'))
all_validation_df.head()

,O2Region,RGIId,Area,Name,Benchmark
0,2,RGI60-01.15253,2.551,NaN,0.0
1,2,RGI60-01.12548,4.314,NaN,0.0
2,2,RGI60-01.15516,4.764,NaN,0.0
3,2,RGI60-01.21721,6.422,Toklat1,0.0
4,2,RGI60-01.12165,7.969,NaN,0.0


In [2]:
# # Trigger the authentication flow.
# ee.Authenticate()

# # Initialize the library.
ee.Initialize()

In [9]:
# load the rgi outlines as GEE feature collection
asset_rgi01_Alaska = ee.FeatureCollection('projects/lzeller/assets/01_rgi60_Alaska')

# option to subset to fewer glaciers
# have run all benchmark glaciers other than taku
validation_df = all_validation_df.copy()
validation_df = validation_df[validation_df['Benchmark']!=1]
validation_df = validation_df[validation_df['O2Region']==4]
# validation_df = validation_df[validation_df['Name']!='Wolverine Glacier']
# validation_df = validation_df[validation_df['Name']!='Taku Glacier']
# validation_df = validation_df[validation_df['Name']=='Wolverine Glacier']

# subset to the validation glaciers
rgi_to_use = validation_df['RGIId'].values.tolist()
rgi_to_use = asset_rgi01_Alaska.filter(ee.Filter.inList('RGIId',rgi_to_use))
rgi_to_use = rgi_to_use.sort('RGIId')

# rgi_to_use = asset_rgi01_Alaska.filter(ee.Filter.inList('Name',['Wolverine Glacier']))

print(len(rgi_to_use.aggregate_array('RGIId').getInfo()))
validation_df.head(10)

10


,O2Region,RGIId,Area,Name,Benchmark
11,4,RGI60-01.10910,2.084,NaN,0.0
12,4,RGI60-01.23606,2.344,NaN,0.0
13,4,RGI60-01.01666,4.243,NaN,0.0
14,4,RGI60-01.09624,4.487,NaN,0.0
15,4,RGI60-01.10255,7.262,NaN,0.0
16,4,RGI60-01.09656,13.791,Langdon Glacier,0.0
17,4,RGI60-01.17784,14.773,Lowell Glacier,0.0
19,4,RGI60-01.09216,18.634,NaN,0.0
21,4,RGI60-01.09798,41.785,Bainbridge Glacier,0.0
22,4,RGI60-01.01743,45.165,Sherman Glacier,0.0


In [10]:
# define lists of band names for each satellite, what they will be renamed to
bands_S2 = ['B2', 'B3', 'B4', 'B8']
bands_L8 = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5']
bands_L7 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4']
bands_L5 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4']
renamed_bands = ['blue', 'green', 'red', 'nir']

# define functions to select correct bands and rename them
def rename_S2(image):
    return image.select(bands_S2).rename(renamed_bands).copyProperties(image, image.propertyNames())
    
def rename_L8(image):
    return image.select(bands_L8).rename(renamed_bands).copyProperties(image, image.propertyNames())

def rename_L7(image):
    return image.select(bands_L7).rename(renamed_bands).copyProperties(image, image.propertyNames())

def rename_L5(image):
    return image.select(bands_L5).rename(renamed_bands).copyProperties(image, image.propertyNames())

# define function to rescale to uint8 image (to minimize storage space used)
# first scale to float reflectance, then rescale to 0-255
def rescale_S2(image):
    return ee.Image(image.select(renamed_bands)
                        .multiply(0.0001).multiply(255)
                        .copyProperties(image, image.propertyNames())
                   ).uint8()

def rescale_LS(image):
    return ee.Image(image.select(renamed_bands)
                        .multiply(2.75e-05).add(-0.2).multiply(255)
                        .copyProperties(image, image.propertyNames())
                   ).uint8()


### Sentinel-2 imagery here:

In [11]:
print("Starting S2")

# iterate through each rgi outline, ordering all Sentinel-2 imagery
date_start = '2018-01-01'
date_end = '2023-01-01'

rgi_names = rgi_to_use.aggregate_array('RGIId').getInfo()
rgi_areas = rgi_to_use.aggregate_array('Area').getInfo()
n_features = len(rgi_names)
rgi_list = rgi_to_use.toList(n_features) 

# now for each glacier, load all sentinel-2 imagery of it
for g in range(0, len(rgi_names)):
#     if g>0: continue
    
    # grab this feature, get rgi name
    rgi_i = ee.Feature(rgi_list.get(g)) 
    rgi_name = rgi_names[g]
    rgi_area = rgi_areas[g]
    print(f"\n{g+1} of {len(rgi_names)} : {rgi_name}")
    if g+1<10: continue
    
    # Load collection of all S2 images
    S2_images = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
                         .filterBounds(rgi_i.geometry())
                         .filterDate(date_start, date_end) # filter by date range
                         .filter(ee.Filter.calendarRange(7,11,'month')) # only in july-november
                         .sort('system:time_start')
                         )

    # define function to merge images that have the same DATATAKE_IDENTIFIER
    def mosaic_DTI(dti):
        
        # subset to only this dti
        subset_ic = S2_images.filter(ee.Filter.eq("DATATAKE_IDENTIFIER",dti))
   
        mosaic = subset_ic.max() # mosaic them
        mosaic = mosaic.set({"DATATAKE_IDENTIFIER":dti})# copy properties
      
        return mosaic
    
    # get list of all the unique image dtis (to mosaic them together)
    dtis = S2_images.aggregate_array("DATATAKE_IDENTIFIER").distinct()
    
    # we will use these dtis for image naming
    names = dtis.getInfo()
    
    # mosaic these together
    s2_swaths = ee.ImageCollection(dtis.map( lambda d : mosaic_DTI(d)) )
    
    # define function to calculate how much coverage each image has
    def calculate_cover(image):
        
        # count how many pixels we see on the glacier
        n_pixels_obs = ee.Number(image.select('B2').reduceRegion(
                                            reducer=ee.Reducer.count(),
                                            geometry=rgi_i.geometry(),
                                            scale=10,
                                            maxPixels=999999999999
                                        ).values().get(0))
        
        # count how many pixels we SHOULD see on the glacier (given full coverage)
        n_pixels_possible = ee.Number(ee.Image(1).reduceRegion(
                                            reducer=ee.Reducer.count(),
                                            geometry=rgi_i.geometry(),
                                            scale=10,
                                            maxPixels=999999999999
                                        ).values().get(0))
        
        cover = n_pixels_obs.divide(n_pixels_possible)
        
        return image.set({'cover':cover})
    
    # calculate coverage of each image, only keep images where we have at least 20% cover
    s2_swaths = s2_swaths.map(lambda image : calculate_cover(image))
    s2_swaths = s2_swaths.filter(ee.Filter.gte('cover', 0.2))
    
    # select bands, rescale
    s2_swaths = s2_swaths.map(lambda image : rename_S2(image))
#     s2_swaths = s2_swaths.map(lambda image : rescale_S2(image))
    
    # sort one last time
    s2_swaths = s2_swaths.sort('DATATAKE_IDENTIFIER')
    
    # get the final image names
    names = s2_swaths.aggregate_array("DATATAKE_IDENTIFIER").getInfo()
    
    # check how many images this is
    n_images = len(names)
    print(n_images)

    # convert image collection to list of images
    S2_img_list = s2_swaths.toList(n_images+1)
    
    # define the scale you will want to export at
    # for the largest glaciers (>40km2) we'll export at slightly coarser resolution to save storage space
    if rgi_area>40: 
        scale=50
    else: 
        scale=30
    
    # iterate through each image and export it
    for i in range(0, n_images):
#         if i>0: continue

        # grab this image
        image_i = ee.Image(S2_img_list.get(i))

        # get the image name
        # image_name = image_i.getInfo().get('properties').get('LANDSAT_PRODUCT_ID') ##########################
        image_name = names[i]

        print(f"Image {i} of {n_images}: {image_name}")

        # define output folder name, file name
        folder_name = 'S2_all_validation_images'
        file_name = f"{rgi_name[:5]}_{rgi_name[6:]}_{image_name[:-3]}"
        
        ### export the image
        run_export=1
        if run_export:
                task = ee.batch.Export.image.toDrive(
                    image = image_i.uint16(),
                    region = rgi_i.geometry().bounds(),
                    folder = folder_name,
                    scale = scale,
                    maxPixels = int(1e13),
                    crs = 'EPSG:3338',
                    crsTransform = [10,0,0,0,-10,0],
                    description = file_name,
                    )

                task.start()
                print('Started')
    g+=1

Starting S2

1 of 10 : RGI60-01.01666
403
Image 0 of 403: GS2A_20180703T210031_015827_N02.06
Started
Image 1 of 403: GS2A_20180706T211511_015870_N02.06
Started
Image 2 of 403: GS2A_20180709T212521_015913_N02.06
Started
Image 3 of 403: GS2A_20180713T210021_015970_N02.06
Started
Image 4 of 403: GS2A_20180716T211511_016013_N02.06
Started
Image 5 of 403: GS2A_20180719T212521_016056_N02.06
Started
Image 6 of 403: GS2A_20180723T210021_016113_N02.06
Started
Image 7 of 403: GS2A_20180726T211511_016156_N02.06
Started
Image 8 of 403: GS2A_20180802T210021_016256_N02.06
Started
Image 9 of 403: GS2A_20180805T211511_016299_N02.06
Started
Image 10 of 403: GS2A_20180808T212521_016342_N02.06
Started
Image 11 of 403: GS2A_20180812T210021_016399_N02.06
Started
Image 12 of 403: GS2A_20180815T211511_016442_N02.06
Started
Image 13 of 403: GS2A_20180822T210021_016542_N02.06
Started
Image 14 of 403: GS2A_20180825T211511_016585_N02.06
Started
Image 15 of 403: GS2A_20180901T210021_016685_N02.06
Started
Image 16

Started
Image 136 of 403: GS2A_20210905T210021_032415_N03.01
Started
Image 137 of 403: GS2A_20210908T211511_032458_N03.01
Started
Image 138 of 403: GS2A_20210911T212521_032501_N03.01
Started
Image 139 of 403: GS2A_20210915T210111_032558_N03.01
Started
Image 140 of 403: GS2A_20210918T211521_032601_N03.01
Started
Image 141 of 403: GS2A_20210921T212531_032644_N03.01
Started
Image 142 of 403: GS2A_20210925T210221_032701_N03.01
Started
Image 143 of 403: GS2A_20210928T211521_032744_N03.01
Started
Image 144 of 403: GS2A_20211001T212531_032787_N03.01
Started
Image 145 of 403: GS2A_20211005T210341_032844_N03.01
Started
Image 146 of 403: GS2A_20211008T211521_032887_N03.01
Started
Image 147 of 403: GS2A_20211015T210441_032987_N03.01
Started
Image 148 of 403: GS2A_20211018T211521_033030_N03.01
Started
Image 149 of 403: GS2A_20211021T212531_033073_N03.01
Started
Image 150 of 403: GS2A_20211025T210551_033130_N03.01
Started
Image 151 of 403: GS2A_20211028T211611_033173_N03.01
Started
Image 152 of 403

Started
Image 271 of 403: GS2B_20191021T210519_013711_N02.08
Started
Image 272 of 403: GS2B_20191024T211539_013754_N02.08
Started
Image 273 of 403: GS2B_20191027T212559_013797_N02.08
Started
Image 274 of 403: GS2B_20191031T210619_013854_N02.08
Started
Image 275 of 403: GS2B_20191103T211639_013897_N02.08
Started
Image 276 of 403: GS2B_20191106T212659_013940_N02.08
Started
Image 277 of 403: GS2B_20191110T210709_013997_N02.08
Started
Image 278 of 403: GS2B_20191113T211729_014040_N02.08
Started
Image 279 of 403: GS2B_20191116T212749_014083_N02.08
Started
Image 280 of 403: GS2B_20191120T210759_014140_N02.08
Started
Image 281 of 403: GS2B_20191123T211809_014183_N02.08
Started
Image 282 of 403: GS2B_20200703T212529_017372_N02.09
Started
Image 283 of 403: GS2B_20200707T210029_017429_N02.09
Started
Image 284 of 403: GS2B_20200710T211519_017472_N02.09
Started
Image 285 of 403: GS2B_20200713T212529_017515_N02.09
Started
Image 286 of 403: GS2B_20200717T210029_017572_N02.09
Started
Image 287 of 403

Started
Image 2 of 297: GS2A_20180713T210021_015970_N02.06
Started
Image 3 of 297: GS2A_20180716T211511_016013_N02.06
Started
Image 4 of 297: GS2A_20180723T210021_016113_N02.06
Started
Image 5 of 297: GS2A_20180726T211511_016156_N02.06
Started
Image 6 of 297: GS2A_20180802T210021_016256_N02.06
Started
Image 7 of 297: GS2A_20180805T211511_016299_N02.06
Started
Image 8 of 297: GS2A_20180812T210021_016399_N02.06
Started
Image 9 of 297: GS2A_20180815T211511_016442_N02.06
Started
Image 10 of 297: GS2A_20180822T210021_016542_N02.06
Started
Image 11 of 297: GS2A_20180825T211511_016585_N02.06
Started
Image 12 of 297: GS2A_20180901T210021_016685_N02.06
Started
Image 13 of 297: GS2A_20180904T211511_016728_N02.06
Started
Image 14 of 297: GS2A_20180911T210031_016828_N02.06
Started
Image 15 of 297: GS2A_20180914T211511_016871_N02.06
Started
Image 16 of 297: GS2A_20180921T210141_016971_N02.06
Started
Image 17 of 297: GS2A_20180924T211511_017014_N02.06
Started
Image 18 of 297: GS2A_20181001T210301_01

Started
Image 139 of 297: GS2A_20221010T210401_038135_N04.00
Started
Image 140 of 297: GS2A_20221013T211521_038178_N04.00
Started
Image 141 of 297: GS2A_20221020T210511_038278_N04.00
Started
Image 142 of 297: GS2A_20221023T211541_038321_N04.00
Started
Image 143 of 297: GS2A_20221030T210621_038421_N04.00
Started
Image 144 of 297: GS2A_20221102T211641_038464_N04.00
Started
Image 145 of 297: GS2A_20221109T210711_038564_N04.00
Started
Image 146 of 297: GS2A_20221112T211731_038607_N04.00
Started
Image 147 of 297: GS2A_20221119T210751_038707_N04.00
Started
Image 148 of 297: GS2A_20221122T211811_038750_N04.00
Started
Image 149 of 297: GS2B_20180701T211509_006890_N02.06
Started
Image 150 of 297: GS2B_20180708T210029_006990_N02.06
Started
Image 151 of 297: GS2B_20180711T211509_007033_N02.06
Started
Image 152 of 297: GS2B_20180718T210029_007133_N02.06
Started
Image 153 of 297: GS2B_20180721T211509_007176_N02.06
Started
Image 154 of 297: GS2B_20180728T210019_007276_N02.06
Started
Image 155 of 297

...

Started
Image 419 of 434: GS2B_20221008T211509_029198_N04.00
Started
Image 420 of 434: GS2B_20221011T212519_029241_N04.00
Started
Image 421 of 434: GS2B_20221015T210439_029298_N04.00
Started
Image 422 of 434: GS2B_20221018T211509_029341_N04.00
Started
Image 423 of 434: GS2B_20221021T212529_029384_N04.00
Started
Image 424 of 434: GS2B_20221025T210539_029441_N04.00
Started
Image 425 of 434: GS2B_20221028T211559_029484_N04.00
Started
Image 426 of 434: GS2B_20221031T212629_029527_N04.00
Started
Image 427 of 434: GS2B_20221104T210639_029584_N04.00
Started
Image 428 of 434: GS2B_20221107T211659_029627_N04.00
Started
Image 429 of 434: GS2B_20221110T212719_029670_N04.00
Started
Image 430 of 434: GS2B_20221114T210729_029727_N04.00
Started
Image 431 of 434: GS2B_20221117T211749_029770_N04.00
Started
Image 432 of 434: GS2B_20221120T212809_029813_N04.00
Started
Image 433 of 434: GS2B_20221124T210809_029870_N04.00
Started

9 of 10 : RGI60-01.17784
447
Image 0 of 447: GS2A_20180702T213531_015813_N02

Started
Image 121 of 447: GS2A_20201016T212531_027782_N02.09
Started
Image 122 of 447: GS2A_20201019T213531_027825_N02.09
Started
Image 123 of 447: GS2A_20201023T211541_027882_N02.09
Started
Image 124 of 447: GS2A_20201026T212611_027925_N02.09
Started
Image 125 of 447: GS2A_20201029T213631_027968_N02.09
Started
Image 126 of 447: GS2A_20201102T211641_028025_N02.09
Started
Image 127 of 447: GS2A_20201105T212701_028068_N02.09
Started
Image 128 of 447: GS2A_20201108T213721_028111_N02.09
Started
Image 129 of 447: GS2A_20201112T211731_028168_N02.09
Started
Image 130 of 447: GS2A_20201115T212751_028211_N02.09
Started
Image 131 of 447: GS2A_20201118T213811_028254_N02.09
Started
Image 132 of 447: GS2A_20201122T211811_028311_N02.09
Started
Image 133 of 447: GS2A_20201125T212831_028354_N02.09
Started
Image 134 of 447: GS2A_20210703T212521_031500_N03.01
Started
Image 135 of 447: GS2A_20210706T213531_031543_N03.01
Started
Image 136 of 447: GS2A_20210710T211521_031600_N03.01
Started
Image 137 of 447

Started
Image 256 of 447: GS2B_20181022T212529_008506_N02.06
Started
Image 257 of 447: GS2B_20181025T213549_008549_N02.06
Started
Image 258 of 447: GS2B_20181029T211609_008606_N02.06
Started
Image 259 of 447: GS2B_20181101T212629_008649_N02.06
Started
Image 260 of 447: GS2B_20181104T213649_008692_N02.06
Started
Image 261 of 447: GS2B_20181108T211659_008749_N02.07
Started
Image 262 of 447: GS2B_20181111T212719_008792_N02.07
Started
Image 263 of 447: GS2B_20181114T213739_008835_N02.07
Started
Image 264 of 447: GS2B_20181118T211749_008892_N02.07
Started
Image 265 of 447: GS2B_20181121T212809_008935_N02.07
Started
Image 266 of 447: GS2B_20181124T213819_008978_N02.07
Started
Image 267 of 447: GS2B_20181128T211829_009035_N02.07
Started
Image 268 of 447: GS2B_20190702T213539_012124_N02.07
Started
Image 269 of 447: GS2B_20190706T211519_012181_N02.07
Started
Image 270 of 447: GS2B_20190709T212529_012224_N02.08
Started
Image 271 of 447: GS2B_20190712T213539_012267_N02.08
Started
Image 272 of 447

Started
Image 391 of 447: GS2B_20211019T213529_024136_N03.01
Started
Image 392 of 447: GS2B_20211023T211539_024193_N03.01
Started
Image 393 of 447: GS2B_20211026T212559_024236_N03.01
Started
Image 394 of 447: GS2B_20211029T213619_024279_N03.01
Started
Image 395 of 447: GS2B_20211102T211639_024336_N03.01
Started
Image 396 of 447: GS2B_20211105T212659_024379_N03.01
Started
Image 397 of 447: GS2B_20211108T213719_024422_N03.01
Started
Image 398 of 447: GS2B_20211112T211729_024479_N03.01
Started
Image 399 of 447: GS2B_20211115T212749_024522_N03.01
Started
Image 400 of 447: GS2B_20211118T213759_024565_N03.01
Started
Image 401 of 447: GS2B_20211122T211809_024622_N03.01
Started
Image 402 of 447: GS2B_20211125T212819_024665_N03.01
Started
Image 403 of 447: GS2B_20220703T212529_027811_N04.00
Started
Image 404 of 447: GS2B_20220706T213539_027854_N04.00
Started
Image 405 of 447: GS2B_20220710T211519_027911_N04.00
Started
Image 406 of 447: GS2B_20220713T212529_027954_N04.00
Started
Image 407 of 447

EEException: Too many tasks already in the queue (3000). Please wait for some of them to complete.

### Landsat 8 imagery here:

In [ ]:
print("Starting L8")

# iterate through each rgi outline, ordering all Landsat 8 imagery
date_start = '2010-01-01'
date_end = '2023-01-01'

rgi_names = rgi_to_use.aggregate_array('RGIId').getInfo()
n_features = len(rgi_names)
rgi_list = rgi_to_use.toList(n_features) 

# now for each glacier, load all L5 imagery of it
for g in range(0, len(rgi_names)):
#     if g<2: continue
    
    # grab this feature, get rgi name
    rgi_i = ee.Feature(rgi_list.get(g)) 
    rgi_name = rgi_names[g]
    print(f"\n{g+1} of {len(rgi_names)} : {rgi_name}")
    
    
    # Load collection of all L8 images
    L8_images = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                         .filterBounds(rgi_i.geometry())
                         .filterDate(date_start, date_end) # filter by date range
                         .filter(ee.Filter.calendarRange(7,11,'month')) # only in july-november
                         .sort('system:time_start')
                         )
    
    # define function to calculate how much coverage each image has
    def calculate_cover(image):
        
        # count how many pixels we see on the glacier
        n_pixels_obs = ee.Number(image.select('SR_B2').reduceRegion(
                                            reducer=ee.Reducer.count(),
                                            geometry=rgi_i.geometry(),
                                            scale=30,
                                            maxPixels=999999999999
                                        ).values().get(0))
        
        # count how many pixels we SHOULD see on the glacier (given full coverage)
        n_pixels_possible = ee.Number(ee.Image(1).reduceRegion(
                                            reducer=ee.Reducer.count(),
                                            geometry=rgi_i.geometry(),
                                            scale=30,
                                            maxPixels=999999999999
                                        ).values().get(0))
        
        cover = n_pixels_obs.divide(n_pixels_possible)
        
        return image.set({'cover':cover})
    
    # calculate coverage of each image, only keep wimages where we have at least 20% cover
    L8_images = L8_images.map(lambda image : calculate_cover(image))
    L8_images = L8_images.filter(ee.Filter.gte('cover', 0.2))
    
    # select bands, rescale
    L8_images = L8_images.map(lambda image : rename_L8(image))
    
    # get list of image names
    names = L8_images.aggregate_array('LANDSAT_PRODUCT_ID').getInfo()

    # check how many images this is
    n_images = len(names)

    # convert image collection to list of images
    L8_img_list = L8_images.toList(n_images+1)
    
    # iterate through each image and export it
    for i in range(0, n_images):
#         if i>0: continue

        # grab this image
        image_i = ee.Image(L8_img_list.get(i))

        # get the image name
        image_name = names[i]

        print(f"Image {i} of {n_images}: {image_name}")

        # define output folder name, file name
        folder_name = 'L8_all_validation_images'
        file_name = f"{rgi_name[:5]}_{rgi_name[6:]}_{image_name}"
        
        # export the image
        run_export=1
        if run_export:
                task = ee.batch.Export.image.toDrive(
                    image = image_i,
                    region = rgi_i.geometry().bounds(),
                    folder = folder_name,
                    scale = 30,
                    maxPixels = int(1e13),
                    crs = 'EPSG:3338',
                    crsTransform = [10,0,0,0,-10,0],
                    description = file_name,
                    )

                task.start()
                print('Started')
    g+=1

### Landsat 9

In [ ]:
print("Starting L9")

# iterate through each rgi outline, ordering all Landsat 8 imagery
date_start = '2010-01-01'
date_end = '2023-01-01'

rgi_names = rgi_to_use.aggregate_array('RGIId').getInfo()
n_features = len(rgi_names)
rgi_list = rgi_to_use.toList(n_features) 

# now for each glacier, load all L5 imagery of it
for g in range(0, len(rgi_names)):
#     if g<2: continue
    
    # grab this feature, get rgi name
    rgi_i = ee.Feature(rgi_list.get(g)) 
    rgi_name = rgi_names[g]
    print(f"\n{g+1} of {len(rgi_names)} : {rgi_name}")
    
    
    # Load collection of all L8 images
    L8_images = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
                         .filterBounds(rgi_i.geometry())
                         .filterDate(date_start, date_end) # filter by date range
                         .filter(ee.Filter.calendarRange(7,11,'month')) # only in july-november
                         .sort('system:time_start')
                         )
    
    # define function to calculate how much coverage each image has
    def calculate_cover(image):
        
        # count how many pixels we see on the glacier
        n_pixels_obs = ee.Number(image.select('SR_B2').reduceRegion(
                                            reducer=ee.Reducer.count(),
                                            geometry=rgi_i.geometry(),
                                            scale=30,
                                            maxPixels=999999999999
                                        ).values().get(0))
        
        # count how many pixels we SHOULD see on the glacier (given full coverage)
        n_pixels_possible = ee.Number(ee.Image(1).reduceRegion(
                                            reducer=ee.Reducer.count(),
                                            geometry=rgi_i.geometry(),
                                            scale=30,
                                            maxPixels=999999999999
                                        ).values().get(0))
        
        cover = n_pixels_obs.divide(n_pixels_possible)
        
        return image.set({'cover':cover})
    
    # calculate coverage of each image, only keep wimages where we have at least 20% cover
    L8_images = L8_images.map(lambda image : calculate_cover(image))
    L8_images = L8_images.filter(ee.Filter.gte('cover', 0.2))
    
    # select bands, rescale
    L8_images = L8_images.map(lambda image : rename_L8(image))
    
    # get list of image names
    names = L8_images.aggregate_array('LANDSAT_PRODUCT_ID').getInfo()

    # check how many images this is
    n_images = len(names)

    # convert image collection to list of images
    L8_img_list = L8_images.toList(n_images+1)
    
    # iterate through each image and export it
    for i in range(0, n_images):
#         if i>0: continue

        # grab this image
        image_i = ee.Image(L8_img_list.get(i))

        # get the image name
        image_name = names[i]

        print(f"Image {i} of {n_images}: {image_name}")

        # define output folder name, file name
        folder_name = 'L9_all_validation_images'
        file_name = f"{rgi_name[:5]}_{rgi_name[6:]}_{image_name}"
        
        # export the image
        run_export=1
        if run_export:
                task = ee.batch.Export.image.toDrive(
                    image = image_i,
                    region = rgi_i.geometry().bounds(),
                    folder = folder_name,
                    scale = 30,
                    maxPixels = int(1e13),
                    crs = 'EPSG:3338',
                    crsTransform = [10,0,0,0,-10,0],
                    description = file_name,
                    )

                task.start()
                print('Started')
    g+=1

### Landsat 7 imagery here:

In [ ]:
print("Starting L7")

# iterate through each rgi outline, ordering all Landsat 7 imagery
date_start = '2000-01-01'
date_end = '2023-01-01'

rgi_names = rgi_to_use.aggregate_array('RGIId').getInfo()
n_features = len(rgi_names)
rgi_list = rgi_to_use.toList(n_features) 

# now for each glacier, load all L5 imagery of it
for g in range(0, len(rgi_names)):
#     if g>0: continue
    
    # grab this feature, get rgi name
    rgi_i = ee.Feature(rgi_list.get(g)) 
    rgi_name = rgi_names[g]
    print(f"\n{g+1} of {len(rgi_names)} : {rgi_name}")
    
    
    
    # Load collection of all L7 images
    L7_images = (ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
                         .filterBounds(rgi_i.geometry())
                         .filterDate(date_start, date_end) # filter by date range
                         .filter(ee.Filter.calendarRange(7,11,'month')) # only in july-november
                         .sort('system:time_start')
                         )
    
    # define function to calculate how much coverage each image has
    def calculate_cover(image):
        
        # count how many pixels we see on the glacier
        n_pixels_obs = ee.Number(image.select('SR_B2').reduceRegion(
                                            reducer=ee.Reducer.count(),
                                            geometry=rgi_i.geometry(),
                                            scale=30,
                                            maxPixels=999999999999
                                        ).values().get(0))
        
        # count how many pixels we SHOULD see on the glacier (given full coverage)
        n_pixels_possible = ee.Number(ee.Image(1).reduceRegion(
                                            reducer=ee.Reducer.count(),
                                            geometry=rgi_i.geometry(),
                                            scale=30,
                                            maxPixels=999999999999
                                        ).values().get(0))
        
        cover = n_pixels_obs.divide(n_pixels_possible)
        
        return image.set({'cover':cover})
    
    # calculate coverage of each image, only keep wimages where we have at least 20% cover
    L7_images = L7_images.map(lambda image : calculate_cover(image))
    L7_images = L7_images.filter(ee.Filter.gte('cover', 0.2))
    
    # select bands, rescale
    L7_images = L7_images.map(lambda image : rename_L7(image))

    # get list of image names
    names = L7_images.aggregate_array('LANDSAT_PRODUCT_ID').getInfo()

    # check how many images this is
    n_images = len(names)

    # convert image collection to list of images
    L7_img_list = L7_images.toList(n_images+1)
    
    # iterate through each image and export it
    for i in range(0, n_images):
#         if i>0: continue

        # grab this image
        image_i = ee.Image(L7_img_list.get(i))

        # get the image name
        image_name = names[i]

        print(f"Image {i} of {n_images}: {image_name}")

        # define output folder name, file name
        folder_name = 'L7_all_validation_images'
        file_name = f"{rgi_name[:5]}_{rgi_name[6:]}_{image_name}"
#         print(file_name)
        
        # export the image
        run_export=1
        if run_export:
                task = ee.batch.Export.image.toDrive(
                    image = image_i,
                    region = rgi_i.geometry().bounds(),
                    folder = folder_name,
                    scale = 30,
                    maxPixels = int(1e13),
                    crs = 'EPSG:3338',
                    crsTransform = [10,0,0,0,-10,0],
                    description = file_name,
                    )

                task.start()
                print("Started")
    g+=1

### Landsat 5 imagery here:

In [ ]:
print("Starting L5")

# iterate through each rgi outline, ordering all Landsat 5 imagery
date_start = '2000-01-01'
date_end = '2023-01-01'

rgi_names = rgi_to_use.aggregate_array('RGIId').getInfo()
n_features = len(rgi_names)
rgi_list = rgi_to_use.toList(n_features) 

# now for each glacier, load all L5 imagery of it
for g in range(0, len(rgi_names)):
#     if g>0: continue
    
    # grab this feature, get rgi name
    rgi_i = ee.Feature(rgi_list.get(g)) 
    rgi_name = rgi_names[g]
    print(f"\n{g+1} of {len(rgi_names)} : {rgi_name}")
    
    
    
    # Load collection of all L5 images
    L5_images = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
                         .filterBounds(rgi_i.geometry())
                         .filterDate(date_start, date_end) # filter by date range
                         .filter(ee.Filter.calendarRange(7,11,'month')) # only in july-november
                         .sort('system:time_start')
                         )
    
    # define function to calculate how much coverage each image has
    def calculate_cover(image):
        
        # count how many pixels we see on the glacier
        n_pixels_obs = ee.Number(image.select('SR_B2').reduceRegion(
                                            reducer=ee.Reducer.count(),
                                            geometry=rgi_i.geometry(),
                                            scale=30,
                                            maxPixels=999999999999
                                        ).values().get(0))
        
        # count how many pixels we SHOULD see on the glacier (given full coverage)
        n_pixels_possible = ee.Number(ee.Image(1).reduceRegion(
                                            reducer=ee.Reducer.count(),
                                            geometry=rgi_i.geometry(),
                                            scale=30,
                                            maxPixels=999999999999
                                        ).values().get(0))
        
        cover = n_pixels_obs.divide(n_pixels_possible)
        
        return image.set({'cover':cover})
    
    # calculate coverage of each image, only keep wimages where we have at least 20% cover
    L5_images = L5_images.map(lambda image : calculate_cover(image))
    L5_images = L5_images.filter(ee.Filter.gte('cover', 0.2))
    
    # select bands, rescale
    L5_images = L5_images.map(lambda image : rename_L5(image))
    
    # get list of image names
    names = L5_images.aggregate_array('LANDSAT_PRODUCT_ID').getInfo()

    # check how many images this is
    n_images = len(names)

    # convert image collection to list of images
    L5_img_list = L5_images.toList(n_images+1)
    
    # iterate through each image and export it
    for i in range(0, n_images):
#         if i>0: continue

        # grab this image
        image_i = ee.Image(L5_img_list.get(i))

        # get the image name
        image_name = names[i]

        print(f"Image {i} of {n_images}: {image_name}")

        # define output folder name, file name
        folder_name = 'L5_all_validation_images'
        file_name = f"{rgi_name[:5]}_{rgi_name[6:]}_{image_name}"
#         print(file_name)
        
        # export the image
        run_export=1
        if run_export:
                task = ee.batch.Export.image.toDrive(
                    image = image_i,
                    region = rgi_i.geometry().bounds(),
                    folder = folder_name,
                    scale = 30,
                    maxPixels = int(1e13),
                    crs = 'EPSG:3338',
                    crsTransform = [10,0,0,0,-10,0],
                    description = file_name,
                    )

                task.start()
                print("Started")
    g+=1